In [1]:
import numpy as np
import torch 
import matplotlib.pyplot as plt
import os
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,random_split
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch import optim
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics.functional import  accuracy 
from pytorch_lightning import loggers as pl_loggers
from torchvision import datasets, transforms
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


C:\Users\mohamedr002\Anaconda3\envs\phd\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [26]:
class HAR_torch_Dataset(Dataset):
    def __init__(self, data,labels, transform= None):
        """Reads source and target sequences from processing file ."""        
        self.input_tensor = (torch.from_numpy(data)).float()
        self.label = torch.LongTensor(labels)
        self.transform =transform
        self.num_total_seqs = len(self.input_tensor)

    def __getitem__(self, index):
        """Returns one data pair (source and target)."""
        input_seq = self.input_tensor[index]
        input_labels = self.label[index]
        if self.transform:
            input_seq = self.transform(input_seq)
        return input_seq, input_labels

    def __len__(self):
        return self.num_total_seqs

def HAR_data_generator(data_dir,save=False):
    # dataloading 
    subject_data= np.loadtxt(f'{data_dir}/train/subject_train.txt')
    # Samples
    train_acc_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_x_train.txt')
    train_acc_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_y_train.txt')
    train_acc_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_z_train.txt')
    train_gyro_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_x_train.txt')
    train_gyro_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_y_train.txt')
    train_gyro_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_z_train.txt')
    train_tot_acc_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_x_train.txt')
    train_tot_acc_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_y_train.txt')
    train_tot_acc_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_z_train.txt')

    test_acc_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_x_test.txt')
    test_acc_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_y_test.txt')
    test_acc_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_z_test.txt')
    test_gyro_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_x_test.txt')
    test_gyro_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_y_test.txt')
    test_gyro_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_z_test.txt')
    test_tot_acc_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_x_test.txt')
    test_tot_acc_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_y_test.txt')
    test_tot_acc_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_z_test.txt')

    # Stacking channels together data 
    train_data= np.stack((train_acc_x,train_acc_y,train_acc_z,
                              train_gyro_x,train_gyro_y,train_gyro_z,
                              train_tot_acc_x, train_tot_acc_y,train_tot_acc_z),axis=1)
    test_data= np.stack((test_acc_x,test_acc_y,test_acc_z,
                              test_gyro_x,test_gyro_y,test_gyro_z,
                              test_tot_acc_x, test_tot_acc_y,test_tot_acc_z),axis=1)
    # labels 
    train_labels=  np.loadtxt(f'{data_dir}/train/y_train.txt')
    train_labels -= np.min(train_labels)
    test_labels=  np.loadtxt(f'{data_dir}/test/y_test.txt')
    test_labels -= np.min(test_labels)


    HAR_dataset_processed= {'train':{'samples':torch.from_numpy(train_data), 'labels':torch.from_numpy(train_labels)},
                               'test':{'samples':torch.from_numpy(test_data), 'labels':torch.from_numpy(test_labels)}}
    if save:
        for mode in ['train', 'test']:
            torch.save(HAR_dataset_processed[mode], f'{mode}.pt')
    return HAR_dataset_processed

In [28]:
data_dir = '../../UCI HAR Dataset'
HAR_data_generator(data_dir,save=True)

{'train': {'samples': tensor([[[ 1.8085e-04,  1.0139e-02,  9.2756e-03,  ..., -1.1475e-03,
            -2.2227e-04,  1.5755e-03],
           [ 1.0767e-02,  6.5795e-03,  8.9289e-03,  ...,  1.7144e-04,
             1.5742e-03,  3.0702e-03],
           [ 5.5561e-02,  5.5125e-02,  4.8405e-02,  ...,  2.6479e-03,
             2.3811e-03, -2.2698e-03],
           ...,
           [ 1.0128e+00,  1.0228e+00,  1.0220e+00,  ...,  1.0184e+00,
             1.0194e+00,  1.0212e+00],
           [-1.2322e-01, -1.2688e-01, -1.2400e-01,  ..., -1.2407e-01,
            -1.2275e-01, -1.2133e-01],
           [ 1.0293e-01,  1.0569e-01,  1.0210e-01,  ...,  1.0039e-01,
             9.9874e-02,  9.4987e-02]],
  
          [[ 1.0938e-03,  4.5501e-03,  2.8792e-03,  ..., -4.6461e-03,
            -2.9413e-03, -1.5988e-03],
           [-4.6876e-03, -7.4879e-03, -8.4300e-03,  ..., -5.4798e-03,
            -2.9875e-03, -7.1920e-03],
           [-2.6860e-02, -2.5098e-02, -2.5975e-02,  ..., -2.9967e-04,
            -4.886

Data Preparation 

In [ ]:
def HAR_data_loader(dataset, batch_size, shuffle, drop_last) :
    # datasets 
    train_dataset = HAR_torch_Dataset(dataset['train']['samples'], dataset['train']['labels'])
    val_dataset = HAR_torch_Dataset(dataset['val']['samples'], dataset['val']['labels'])
    test_dataset = HAR_torch_Dataset(dataset['test']['samples'], dataset['test']['labels'])
    # dataloaders
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    valid_dl = DataLoader(val_dataset, batch_size=10, shuffle=False, drop_last=False)
    test_dl = DataLoader(test_dataset, batch_size=10, shuffle=False, drop_last=False)

    return train_dl, valid_dl, test_dl

In [29]:
a=torch.load('test.pt')

In [30]:
a['samples'].shape

torch.Size([2947, 9, 128])